In [293]:
#importing all the required libraries

import numpy as np
import pandas as pd
from rdkit.Chem import PandasTools
import pandas as pd
import os
from rdkit import RDConfig
from rdkit import Chem
from rdkit.Chem import AllChem
import math
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error,r2_score,accuracy_score
import pickle

In [ ]:
!pip install selfies


In [ ]:
!pip install rdkit-pypi

In [ ]:
#loaded the MCL1 Target dataset having uniprot id "Q07820"

df=pd.read_csv("/content/MCL1-Target_data_2.csv")

In [ ]:
df.head()

,Molecule ChEMBL ID,Molecule Name,Molecule Max Phase,Molecular Weight,#RO5 Violations,AlogP,Compound Key,Smiles,Standard Type,Standard Relation,...,Target Organism,Target Type,Document ChEMBL ID,Source ID,Source Description,Document Journal,Document Year,Cell ChEMBL ID,Properties,Action Type
0,CHEMBL1416266,NaN,None,354.50,0,4.31,SID22404759,CSc1cnn(Cc2ccccc2)c(=O)c1SCc1ccccc1,IC50,'>',...,Homo sapiens,SINGLE PROTEIN,CHEMBL1201862,7.0,PubChem BioAssays,NaN,NaN,None,NaN,NaN
1,CHEMBL1560760,NaN,None,277.31,0,1.31,SID24784448,CC1=NN(C(N)=S)C(=O)C1N=Nc1ccc(O)cc1,IC50,'=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL1201862,7.0,PubChem BioAssays,NaN,NaN,None,NaN,NaN
2,CHEMBL1504470,NaN,None,346.45,0,3.78,SID16952999,CCN(c1ccc(NC(=O)COC(=O)c2cccs2)cc1)C(C)C,IC50,'=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL1201862,7.0,PubChem BioAssays,NaN,NaN,None,NaN,NaN
3,CHEMBL1307244,NaN,None,337.36,0,3.48,SID17505351,N=c1oc2cc(O)ccc2cc1C(=O)Nc1nc2ccccc2s1,IC50,'=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL1201862,7.0,PubChem BioAssays,NaN,NaN,None,NaN,NaN
4,CHEMBL1979530,NaN,None,295.37,0,3.43,SID24836107,Oc1ccc(/C=N/n2cc(-c3ccccc3)nc2S)cc1,IC50,'=',...,Homo sapiens,SINGLE PROTEIN,CHEMBL1201862,7.0,PubChem BioAssays,NaN,NaN,None,NaN,NaN


In [ ]:
df.shape

(1865, 46)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1865 entries, 0 to 1864
Data columns (total 46 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Molecule ChEMBL ID          1865 non-null   object 
 1   Molecule Name               66 non-null     object 
 2   Molecule Max Phase          1861 non-null   object 
 3   Molecular Weight            1861 non-null   float64
 4   #RO5 Violations             1861 non-null   object 
 5   AlogP                       1861 non-null   object 
 6   Compound Key                1861 non-null   object 
 7   Smiles                      1838 non-null   object 
 8   Standard Type               1840 non-null   object 
 9   Standard Relation           1828 non-null   object 
 10  Standard Value              1831 non-null   float64
 11  Standard Units              1831 non-null   object 
 12  pChEMBL Value               1569 non-null   float64
 13  Data Validity Comment       28 no

In [ ]:
df['Standard Type'].unique()

array(['IC50', nan], dtype=object)

In [ ]:
df['pChEMBL Value'].head()

0     NaN
1    5.01
2    5.85
3    5.49
4    4.78
Name: pChEMBL Value, dtype: float64

In [ ]:
#take a new data frame with required columns i.e; Smiles and pChemble values
data_frame=df[["Smiles","pChEMBL Value"]]
data_frame.head()

In [ ]:
#pChemble value is the values of standard type i.e; IC50 (so renamed the column name of pChemble as IC50)
data_frame.rename(columns={'pChEMBL Value':'IC50'},inplace=True)
data_frame.head()

<ipython-input-212-8663c7e1bfba>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame.rename(columns={'pChEMBL Value':'IC50'},inplace=True)


In [ ]:
# Data pre processing

#Check for the null values
data_frame.isnull().sum()

Smiles     27
IC50      296
dtype: int64

In [ ]:
#Drop the rows having null values
data_frame.dropna(axis=0,inplace=True)
data_frame.isnull().sum()

<ipython-input-215-6c3acd0ff8b1>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame.dropna(axis=0,inplace=True)


In [ ]:
data_frame.shape

(1569, 2)

In [ ]:
#Validating smiles

def check_smile(smi):
  mol=Chem.MolFromSmiles(smi)
  return mol

data_frame["mol"]=data_frame['Smiles'].apply(check_smile)
data_frame.head()

<ipython-input-218-d94e0d0dcf8c>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame["mol"]=data_frame['Smiles'].apply(check_smile)


,Smiles,IC50,mol
1,CC1=NN(C(N)=S)C(=O)C1N=Nc1ccc(O)cc1,5.01,<rdkit.Chem.rdchem.Mol object at 0x78c705bb0ac0>
2,CCN(c1ccc(NC(=O)COC(=O)c2cccs2)cc1)C(C)C,5.85,<rdkit.Chem.rdchem.Mol object at 0x78c705bb0a50>
3,N=c1oc2cc(O)ccc2cc1C(=O)Nc1nc2ccccc2s1,5.49,<rdkit.Chem.rdchem.Mol object at 0x78c705bb0c80>
4,Oc1ccc(/C=N/n2cc(-c3ccccc3)nc2S)cc1,4.78,<rdkit.Chem.rdchem.Mol object at 0x78c705bb0c10>
5,O=C(O)CCn1cc(C=C2C(=O)NC(=S)NC2=O)c(-c2cccc(Br...,5.62,<rdkit.Chem.rdchem.Mol object at 0x78c705bb0cf0>


In [ ]:
# reset the index to avoid errors while iterating through data frame
data_frame.reset_index(drop=True,inplace=True)
data_frame.head()

,Smiles,IC50,mol
0,CC1=NN(C(N)=S)C(=O)C1N=Nc1ccc(O)cc1,5.01,<rdkit.Chem.rdchem.Mol object at 0x78c705bb0ac0>
1,CCN(c1ccc(NC(=O)COC(=O)c2cccs2)cc1)C(C)C,5.85,<rdkit.Chem.rdchem.Mol object at 0x78c705bb0a50>
2,N=c1oc2cc(O)ccc2cc1C(=O)Nc1nc2ccccc2s1,5.49,<rdkit.Chem.rdchem.Mol object at 0x78c705bb0c80>
3,Oc1ccc(/C=N/n2cc(-c3ccccc3)nc2S)cc1,4.78,<rdkit.Chem.rdchem.Mol object at 0x78c705bb0c10>
4,O=C(O)CCn1cc(C=C2C(=O)NC(=S)NC2=O)c(-c2cccc(Br...,5.62,<rdkit.Chem.rdchem.Mol object at 0x78c705bb0cf0>


In [ ]:
#Check whether there are any invalid smiles
count=0
for i in range(len(data_frame)):
  if data_frame["mol"][i]==None:
    count+=1
print(count)

0


In [ ]:
#getting finger prints from smiles

data_frame["FingerPrints"]=data_frame['Smiles'].apply(lambda x:AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(x),2))
data_frame.head()

<ipython-input-223-d183e6cbdd87>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame["FingerPrints"]=data_frame['Smiles'].apply(lambda x:AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(x),2))


,Smiles,IC50,mol,FingerPrints
0,CC1=NN(C(N)=S)C(=O)C1N=Nc1ccc(O)cc1,5.01,<rdkit.Chem.rdchem.Mol object at 0x78c705bb0ac0>,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,CCN(c1ccc(NC(=O)COC(=O)c2cccs2)cc1)C(C)C,5.85,<rdkit.Chem.rdchem.Mol object at 0x78c705bb0a50>,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
2,N=c1oc2cc(O)ccc2cc1C(=O)Nc1nc2ccccc2s1,5.49,<rdkit.Chem.rdchem.Mol object at 0x78c705bb0c80>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,Oc1ccc(/C=N/n2cc(-c3ccccc3)nc2S)cc1,4.78,<rdkit.Chem.rdchem.Mol object at 0x78c705bb0c10>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,O=C(O)CCn1cc(C=C2C(=O)NC(=S)NC2=O)c(-c2cccc(Br...,5.62,<rdkit.Chem.rdchem.Mol object at 0x78c705bb0cf0>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
#transformtion of IC50 to pIC50 values

#pIC50 = -log (IC50*10**-9)

def get_pIC50(ic):
  return -math.log(ic*math.pow(10,-9),10)

data_frame['pIC50']=data_frame['IC50'].apply(get_pIC50)
data_frame.head()

<ipython-input-225-e8399966afd1>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_frame['pIC50']=data_frame['IC50'].apply(get_pIC50)


,Smiles,IC50,mol,FingerPrints,pIC50
0,CC1=NN(C(N)=S)C(=O)C1N=Nc1ccc(O)cc1,5.01,<rdkit.Chem.rdchem.Mol object at 0x78c705bb0ac0>,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",8.300162
1,CCN(c1ccc(NC(=O)COC(=O)c2cccs2)cc1)C(C)C,5.85,<rdkit.Chem.rdchem.Mol object at 0x78c705bb0a50>,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ...",8.232844
2,N=c1oc2cc(O)ccc2cc1C(=O)Nc1nc2ccccc2s1,5.49,<rdkit.Chem.rdchem.Mol object at 0x78c705bb0c80>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",8.260428
3,Oc1ccc(/C=N/n2cc(-c3ccccc3)nc2S)cc1,4.78,<rdkit.Chem.rdchem.Mol object at 0x78c705bb0c10>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",8.320572
4,O=C(O)CCn1cc(C=C2C(=O)NC(=S)NC2=O)c(-c2cccc(Br...,5.62,<rdkit.Chem.rdchem.Mol object at 0x78c705bb0cf0>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",8.250264


In [ ]:
# Training the data to a model by taking fingerprints of molecules as input and IC50 values as output

X=np.array(list(data_frame['FingerPrints']))
Y=np.array(data_frame['IC50'])

In [ ]:
#split the data using train_test_split module
x_train,x_test,y_train,y_test=train_test_split(X,Y,test_size=0.2,random_state=42)

In [ ]:
y_train

array([9.22, 5.47, 5.14, ..., 4.54, 5.82, 5.68])

In [ ]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)


(1255, 2048)
(314, 2048)
(1255,)
(314,)


In [ ]:
#Training using Linear Regression

lr_model=LinearRegression()
lr_model.fit(x_train,y_train)
yhat=lr_model.predict(x_test)
mse=mean_squared_error(y_test,yhat)
print("Accuracy = ",lr_model.score(x_test,y_test))
print("Mean squared Error = ",mse)

Accuracy =  -5.207387915350483e+24
Mean squared Error =  4.2078857629231767e+24


In [ ]:
#Using Random forest regressor for training

rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(x_train, y_train)
rf_predictions = rf_model.predict(x_test)
rf_mse = mean_squared_error(y_test, rf_predictions)
rf_rmse = np.sqrt(rf_mse)
rf_r_squared = rf_model.score(x_test, y_test)

mse = mean_squared_error(y_test, rf_predictions)
print(f'Mean Squared Error: {mse}')
print(f'Random Forest Root Mean Squared Error: {rf_rmse}')
print(f'Random Forest R-squared: {rf_r_squared}')

Mean Squared Error: 0.22982993052834721
Random Forest Root Mean Squared Error: 0.4794058098608602
Random Forest R-squared: 0.7155783996408349


In [ ]:
#Using Gradient Boost Regressor for training

gb_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gb_model.fit(x_train, y_train)
gb_predictions = gb_model.predict(x_test)
gb_mse = mean_squared_error(y_test, gb_predictions)
gb_rmse = np.sqrt(gb_mse)
gb_r_squared = gb_model.score(x_test, y_test)

print(f'Mean Squared Error: {gb_mse}')
print(f'Gradient Boosting Root Mean Squared Error: {gb_rmse}')
print(f'Gradient Boosting R-squared: {gb_r_squared}')

Mean Squared Error: 0.23559087018087246
Gradient Boosting Root Mean Squared Error: 0.48537703919826336
Gradient Boosting R-squared: 0.708449059820834


In [ ]:
pip install xgboost

In [ ]:
#Using xgBoost for training

xgb_model = xgb.XGBRegressor(objective ='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
xgb_model.fit(x_train, y_train)
xgb_predictions = xgb_model.predict(x_test)
xgb_mse = mean_squared_error(y_test, xgb_predictions)
xgb_rmse = np.sqrt(xgb_mse)
xgb_r_squared = xgb_model.score(x_test, y_test)

print(f'Mean Squared Error: {xgb_mse}')
print(f'XGBoost Root Mean Squared Error: {xgb_rmse}')
print(f'XGBoost R-squared: {xgb_r_squared}')


Mean Squared Error: 0.24590867210633363
XGBoost Root Mean Squared Error: 0.49589179475600687
XGBoost R-squared: 0.6956804629323334


In [ ]:
#Saving the model which have the better accuracy

pickle.dump(rf_model, open('random_forest_model_using_MCL1_target.pkl', 'wb'))

In [309]:
#IC50 Activity prediction using better model

#print("testing smile = ",data_frame['Smiles'][0])
print("Actual value = ",data_frame['IC50'][0])

new_smiles = "CC1=NN(C(N)=S)C(=O)C1N=Nc1ccc(O)cc1"
new_fingerprint = AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(new_smiles), 2)
predicted_IC50 = rf_model.predict(np.array([new_fingerprint]))
print("predicted value = ",predicted_IC50)

Actual value =  CC1=NN(C(N)=S)C(=O)C1N=Nc1ccc(O)cc1
Actual value =  5.01
predicted value =  [5.1813]


In [ ]:
#IC50 prediction on unseen data

In [ ]:
#The unseen data we have taken here is in sdf format
# Code for loading sdf file  ,pre-processing and saving it as csv file

sdfFile = os.path.join(RDConfig.RDDataDir,'/content/Discovery_Diversity_Set_50240cmpds_DDS-50_20221206.sdf')

In [ ]:
data_frame = PandasTools.LoadSDF(sdfFile,smilesName='SMILES',molColName='Molecule',
           includeFingerprints=True)

In [ ]:
data_frame.head()

,Catalog ID,MW,MW (desalted),ClogP,logS,HBD,HBA,TPSA,RotBonds,ID,SMILES,Molecule
0,Z2966541131,262.351,262.351,0.872,-1.746,1,3,61.88,3,,CC1(C)CN(C(=O)CCc2c[nH]nn2)[C@@H]2CCC[C@@H]21,<rdkit.Chem.rdchem.Mol object at 0x7c45b29d0580>
1,Z734259234,288.391,288.391,2.975,-3.8,1,5,60.94,4,,CC(C)(C)CCN1CCN(c2ncnc3nc[nH]c23)CC1,<rdkit.Chem.rdchem.Mol object at 0x7c45b29d0350>
2,Z3016333445,295.288,295.288,0.908,-2.892,1,3,63.05,1,,Cc1nn(C)c2ncc(NC(=O)N3C[C@@H](F)[C@@H](F)C3)cc12,<rdkit.Chem.rdchem.Mol object at 0x7c45b29d03c0>
3,Z2644709353,342.435,342.435,2.236,-4.558,1,4,74.66,2,,CN(C(=O)c1cc2c(=O)[nH]cnc2cn1)C1CC(C)(C)CC(C)(...,<rdkit.Chem.rdchem.Mol object at 0x7c45b29d0660>
4,Z3290701816,308.396,308.396,2.593,-4.63,0,3,61.29,6,,COCC1(CSc2nnc(-c3ccoc3C)o2)CCCC1,<rdkit.Chem.rdchem.Mol object at 0x7c45b29d07b0>


In [ ]:
data_frame.shape

(50240, 12)

In [ ]:
data_frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 50240 entries, 0 to 50239
Data columns (total 12 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Catalog ID     50240 non-null  object
 1   MW             50240 non-null  object
 2   MW (desalted)  50240 non-null  object
 3   ClogP          50240 non-null  object
 4   logS           50189 non-null  object
 5   HBD            50240 non-null  object
 6   HBA            50240 non-null  object
 7   TPSA           50240 non-null  object
 8   RotBonds       50240 non-null  object
 9   ID             50240 non-null  object
 10  SMILES         50240 non-null  object
 11  Molecule       50240 non-null  object
dtypes: object(12)
memory usage: 5.0+ MB


In [ ]:
data_frame.isnull().sum()

Catalog ID        0
MW                0
MW (desalted)     0
ClogP             0
logS             51
HBD               0
HBA               0
TPSA              0
RotBonds          0
ID                0
SMILES            0
Molecule          0
dtype: int64

In [ ]:
data_frame.to_csv("ChemBioSM1.csv")

In [ ]:
#Loading the model to predict IC50 values on the unseen data
with open("/content/random_forest_model_using_MCL1_target.pkl",'rb') as f:
  model=pickle.load(f)

In [ ]:
model

RandomForestRegressor(random_state=42)

In [ ]:
#Load the data
new_data_frame=pd.read_csv("/content/ChemBioSM1.csv")

In [ ]:
new_data_frame.head()

,Unnamed: 0,Catalog ID,MW,MW (desalted),ClogP,logS,HBD,HBA,TPSA,RotBonds,ID,SMILES,Molecule
0,0,Z2966541131,262.351,262.351,0.872,-1.746,1,3,61.88,3,NaN,CC1(C)CN(C(=O)CCc2c[nH]nn2)[C@@H]2CCC[C@@H]21,<rdkit.Chem.rdchem.Mol object at 0x7c45b29d0580>
1,1,Z734259234,288.391,288.391,2.975,-3.800,1,5,60.94,4,NaN,CC(C)(C)CCN1CCN(c2ncnc3nc[nH]c23)CC1,<rdkit.Chem.rdchem.Mol object at 0x7c45b29d0350>
2,2,Z3016333445,295.288,295.288,0.908,-2.892,1,3,63.05,1,NaN,Cc1nn(C)c2ncc(NC(=O)N3C[C@@H](F)[C@@H](F)C3)cc12,<rdkit.Chem.rdchem.Mol object at 0x7c45b29d03c0>
3,3,Z2644709353,342.435,342.435,2.236,-4.558,1,4,74.66,2,NaN,CN(C(=O)c1cc2c(=O)[nH]cnc2cn1)C1CC(C)(C)CC(C)(...,<rdkit.Chem.rdchem.Mol object at 0x7c45b29d0660>
4,4,Z3290701816,308.396,308.396,2.593,-4.630,0,3,61.29,6,NaN,COCC1(CSc2nnc(-c3ccoc3C)o2)CCCC1,<rdkit.Chem.rdchem.Mol object at 0x7c45b29d07b0>


In [ ]:
#pre-processing
#check for null values (here we need to focus on SMILES, as there are no null values in smiles we are not dropping the rows)
new_data_frame.isnull().sum()

Unnamed: 0           0
Catalog ID           0
MW                   0
MW (desalted)        0
ClogP                0
logS                51
HBD                  0
HBA                  0
TPSA                 0
RotBonds             0
ID               50240
SMILES               0
Molecule             0
dtype: int64

In [ ]:
#converting smiles into fingerprints
new_data_frame["FingerPrints"]=new_data_frame['SMILES'].apply(lambda x:AllChem.GetMorganFingerprintAsBitVect(Chem.MolFromSmiles(x),2))

In [ ]:
#loading the fingerprints into 'x' and given to model to predict IC50 values
x=new_data_frame['FingerPrints'].to_list()
new_data_predictions=model.predict(np.array(x))
print(new_data_predictions)
new_data_frame['IC50-Predictions']=new_data_predictions

[5.414575 5.163    6.04445  ... 5.6332   5.3531   5.421   ]


In [ ]:
new_data_frame.columns

Index(['Unnamed: 0', 'Catalog ID', 'MW', 'MW (desalted)', 'ClogP', 'logS',
       'HBD', 'HBA', 'TPSA', 'RotBonds', 'ID', 'SMILES', 'Molecule',
       'FingerPrints', 'IC50-Predictions', 'pIC50'],
      dtype='object')

In [ ]:
#converting IC50 to pIC50 values
import math
def get_pIC50(ic):
  return -math.log(ic*math.pow(10,-9),10)

new_data_frame['pIC50']=new_data_frame['IC50-Predictions'].apply(get_pIC50)
new_data_frame.head()

,Unnamed: 0,Catalog ID,MW,MW (desalted),ClogP,logS,HBD,HBA,TPSA,RotBonds,ID,SMILES,Molecule,FingerPrints,IC50-Predictions,pIC50
0,0,Z2966541131,262.351,262.351,0.872,-1.746,1,3,61.88,3,NaN,CC1(C)CN(C(=O)CCc2c[nH]nn2)[C@@H]2CCC[C@@H]21,<rdkit.Chem.rdchem.Mol object at 0x7c45b29d0580>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",5.414575,8.266436
1,1,Z734259234,288.391,288.391,2.975,-3.800,1,5,60.94,4,NaN,CC(C)(C)CCN1CCN(c2ncnc3nc[nH]c23)CC1,<rdkit.Chem.rdchem.Mol object at 0x7c45b29d0350>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",5.163000,8.287098
2,2,Z3016333445,295.288,295.288,0.908,-2.892,1,3,63.05,1,NaN,Cc1nn(C)c2ncc(NC(=O)N3C[C@@H](F)[C@@H](F)C3)cc12,<rdkit.Chem.rdchem.Mol object at 0x7c45b29d03c0>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",6.044450,8.218643
3,3,Z2644709353,342.435,342.435,2.236,-4.558,1,4,74.66,2,NaN,CN(C(=O)c1cc2c(=O)[nH]cnc2cn1)C1CC(C)(C)CC(C)(...,<rdkit.Chem.rdchem.Mol object at 0x7c45b29d0660>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",5.273800,8.277876
4,4,Z3290701816,308.396,308.396,2.593,-4.630,0,3,61.29,6,NaN,COCC1(CSc2nnc(-c3ccoc3C)o2)CCCC1,<rdkit.Chem.rdchem.Mol object at 0x7c45b29d07b0>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",5.390833,8.268344


In [ ]:
#Lipinski rule of 5 prediction

new_data_frame=new_data_frame[new_data_frame['MW']<500]
new_data_frame.shape

(50234, 16)

In [ ]:
#https://www.google.com/search?client=opera&q=clogp+to+logp&sourceid=opera&ie=UTF-8&oe=UTF-8

new_data_frame=new_data_frame[new_data_frame['ClogP']<5]
new_data_frame.shape

(50234, 16)

In [ ]:
new_data_frame=new_data_frame[new_data_frame['HBD']<5]
new_data_frame.shape

(50227, 16)

In [ ]:
new_data_frame=new_data_frame[new_data_frame['HBA']<10]
new_data_frame.shape

(50227, 16)

In [ ]:
#Filter molecules with IC50 > 7
new_data_frame=new_data_frame[new_data_frame['pIC50']>7]
new_data_frame.shape

(50227, 16)

In [ ]:
#Rank the molecules in decreasing order of activity
new_data_frame = new_data_frame.sort_values(by=['pIC50'], ascending=False)

In [310]:
new_data_frame['pIC50'].head(50).to_frame()

,pIC50
19326,8.410553
47032,8.405449
48616,8.393812
29786,8.392563
38994,8.391941
10786,8.390416
39009,8.383566
33565,8.382601
39108,8.381655
35151,8.378533


In [ ]:
new_data_frame.columns

Index(['Unnamed: 0', 'Catalog ID', 'MW', 'MW (desalted)', 'ClogP', 'logS',
       'HBD', 'HBA', 'TPSA', 'RotBonds', 'ID', 'SMILES', 'Molecule',
       'FingerPrints', 'IC50-Predictions', 'pIC50'],
      dtype='object')

In [ ]:
new_data_frame.drop(columns=['Unnamed: 0'],inplace=True)

In [ ]:
new_data_frame.head(20)

,Catalog ID,MW,MW (desalted),ClogP,logS,HBD,HBA,TPSA,RotBonds,ID,SMILES,Molecule,FingerPrints,IC50-Predictions,pIC50
19326,Z741101748,346.361,346.361,2.014,-3.076,3,7,132.18,6,NaN,N#Cc1cccnc1NCCNS(=O)(=O)c1ccc(C(=O)O)cc1,<rdkit.Chem.rdchem.Mol object at 0x7c45ad81b760>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3.885500,8.410553
47032,Z2273620484,257.331,257.331,1.007,-2.550,1,4,60.15,2,NaN,N#Cc1cccnc1N1CCCC1C1CCCC1O,<rdkit.Chem.rdchem.Mol object at 0x7c45aae58190>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",3.931433,8.405449
48616,Z3063556418,304.270,304.270,1.855,-3.204,0,4,52.81,2,NaN,N#Cc1cccnc1N1CCc2ncc(C(F)(F)F)cc2C1,<rdkit.Chem.rdchem.Mol object at 0x7c45aac97290>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4.038200,8.393812
29786,Z3016332987,239.276,239.276,0.780,-1.764,1,4,66.53,2,NaN,N#Cc1cccnc1NC1CCn2nccc2C1,<rdkit.Chem.rdchem.Mol object at 0x7c45ac7551c0>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4.049833,8.392563
38994,Z192635940,284.356,284.356,1.333,-3.520,0,4,60.23,2,NaN,N#Cc1cccnc1N1CCN(C(=O)C2CCCC2)CC1,<rdkit.Chem.rdchem.Mol object at 0x7c45abaa2c70>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4.055633,8.391941
10786,Z2364627442,394.513,394.513,2.761,-4.884,2,5,98.28,3,NaN,N#Cc1cccnc1N1CCN(C(=O)NCC2(N)C3CC4CC(C3)CC2C4)CC1,<rdkit.Chem.rdchem.Mol object at 0x7c45ae3beea0>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4.069900,8.390416
39009,Z2093402204,243.304,243.304,1.334,-2.381,0,4,49.15,2,NaN,N#Cc1cccnc1N1CCCC1C1CCCO1,<rdkit.Chem.rdchem.Mol object at 0x7c45abaa3a70>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4.134600,8.383566
33565,Z2945453085,257.153,257.331,1.763,-2.646,0,4,49.15,2,NaN,N#Cc1cccnc1N1CCCC1C1CCCCO1,<rdkit.Chem.rdchem.Mol object at 0x7c45ac2c65e0>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4.143800,8.382601
39108,Z2762781238,301.340,301.340,0.911,-2.460,1,6,86.45,3,NaN,N#Cc1cccnc1N1CCC([C@@H]2CC[C@H](C(=O)O)O2)CC1,<rdkit.Chem.rdchem.Mol object at 0x7c45abad5af0>,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4.152833,8.381655
35151,Z2827778391,365.432,365.432,-0.158,-2.681,2,6,98.87,4,NaN,N#Cc1ccccc1N1CCC(NC(=O)c2cn(C3CCNC3)nn2)CC1,<rdkit.Chem.rdchem.Mol object at 0x7c45ac119380>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",4.182800,8.378533


In [ ]:
#saved the data after applying all the filters mentioned in requirements
new_data_frame.to_csv("ChemBioSM1_Flitered_MCL1.csv")